# Chapter 7 — Data Transformation (Python / pandas)

This Colab notebook compiles the chapter's code examples **in the same pedagogical order**: merging, cleaning, type conversion, feature engineering, outliers, and aggregation/pivot tables.

## 0) Setup

In [1]:
# Core libraries used across examples
import pandas as pd
import numpy as np


## 1) Merge data from multiple sources

### 1.1 Concatenate (pd.concat) — vertical stacking

In [3]:
import pandas as pd

# DataFrame 1
data1 = {'Employee ID': [1, 2], 'Name': ['Ahmed', 'Fatimah']}
df1 = pd.DataFrame(data1)

# DataFrame 2
data2 = {'Employee ID': [3, 4], 'Name': ['Khalid', 'Layla']}
df2 = pd.DataFrame(data2)

print('DataFrame 1:')
print(df1)
print('DataFrame 2:')
print(df2)

# Vertical concatenation
concatenated_data = pd.concat([df1, df2], ignore_index=True)
print('Concatenated (pd.concat):')
print(concatenated_data)


DataFrame 1:
   Employee ID     Name
0            1    Ahmed
1            2  Fatimah
DataFrame 2:
   Employee ID    Name
0            3  Khalid
1            4   Layla
Concatenated (pd.concat):
   Employee ID     Name
0            1    Ahmed
1            2  Fatimah
2            3   Khalid
3            4    Layla


### 1.2 Table joins (pd.merge) — inner/left/right/outer

In [5]:
import pandas as pd

# Table 1: Employee Information
employee_info = pd.DataFrame({
    'Employee ID': [1, 2, 3],
    'Name': ['Ahmed', 'Fatimah', 'Khalid']
})

# Table 2: Job Information
job_info = pd.DataFrame({
    'Employee ID': [2, 3, 4],
    'Job Title': ['Data Analyst', 'Systems Developer', 'Project Manager']
})

print('Employee info:')
print(employee_info)
print('Job info:')
print(job_info)

# Inner join
merged_inner = pd.merge(employee_info, job_info, on='Employee ID', how='inner')
print('Inner join:')
print(merged_inner)

# Left join
merged_left = pd.merge(employee_info, job_info, on='Employee ID', how='left')
print('Left join:')
print(merged_left)

# Right join
merged_right = pd.merge(employee_info, job_info, on='Employee ID', how='right')
print('Right join:')
print(merged_right)

# Outer join
merged_outer = pd.merge(employee_info, job_info, on='Employee ID', how='outer')
print('Outer join:')
print(merged_outer)


Employee info:
   Employee ID     Name
0            1    Ahmed
1            2  Fatimah
2            3   Khalid
Job info:
   Employee ID          Job Title
0            2       Data Analyst
1            3  Systems Developer
2            4    Project Manager
Inner join:
   Employee ID     Name          Job Title
0            2  Fatimah       Data Analyst
1            3   Khalid  Systems Developer
Left join:
   Employee ID     Name          Job Title
0            1    Ahmed                NaN
1            2  Fatimah       Data Analyst
2            3   Khalid  Systems Developer
Right join:
   Employee ID     Name          Job Title
0            2  Fatimah       Data Analyst
1            3   Khalid  Systems Developer
2            4      NaN    Project Manager
Outer join:
   Employee ID     Name          Job Title
0            1    Ahmed                NaN
1            2  Fatimah       Data Analyst
2            3   Khalid  Systems Developer
3            4      NaN    Project Manager


## 2) Data cleaning

### 2.1 Remove rows with missing values (df.dropna)

In [6]:
import pandas as pd
import numpy as np

data = {
    'Employee ID': [1, 2, 3, 4, 5],
    'Name': ['Layla', 'Khalid', np.nan, 'Fatimah', 'Ahmed'],
    'Salary': [50000, 60000, 75000, np.nan, 55000],
    'Department': ['IT', 'HR', 'Finance', 'IT', np.nan]
}

df = pd.DataFrame(data)
print('Original DataFrame:')
print(df)

# Drop rows containing ANY missing values
df_cleaned = df.dropna()
print('After dropna():')
print(df_cleaned)


Original DataFrame:
   Employee ID     Name   Salary Department
0            1    Layla  50000.0         IT
1            2   Khalid  60000.0         HR
2            3      NaN  75000.0    Finance
3            4  Fatimah      NaN         IT
4            5    Ahmed  55000.0        NaN
After dropna():
   Employee ID    Name   Salary Department
0            1   Layla  50000.0         IT
1            2  Khalid  60000.0         HR


### 2.2 Imputation (fill missing values) — numeric mean + categorical placeholders

In [7]:
import pandas as pd
import numpy as np

data = {
    'Employee ID': [1, 2, 3, 4, 5],
    'Name': ['Layla', 'Khalid', np.nan, 'Fatimah', 'Ahmed'],
    'Salary': [50000, 60000, 75000, np.nan, 55000],
    'Department': ['IT', 'HR', 'Finance', 'IT', np.nan]
}

df = pd.DataFrame(data)

# Numeric: fill with mean
df['Salary'].fillna(df['Salary'].mean(), inplace=True)

# Categorical: fill with explicit labels
df['Name'].fillna('Unknown', inplace=True)
df['Department'].fillna('Unspecified', inplace=True)

print('After imputations:')
print(df)


After imputations:
   Employee ID     Name   Salary   Department
0            1    Layla  50000.0           IT
1            2   Khalid  60000.0           HR
2            3  Unknown  75000.0      Finance
3            4  Fatimah  60000.0           IT
4            5    Ahmed  55000.0  Unspecified


/tmp/ipython-input-317968572.py:14: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['Salary'].fillna(df['Salary'].mean(), inplace=True)
/tmp/ipython-input-317968572.py:17: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', 

### 2.3 Forward fill (ffill) — useful for time-ordered data

In [9]:
import pandas as pd
import numpy as np

data = {
    'Date': pd.to_datetime(['2023-01-01', '2023-01-02', '2023-01-03', '2023-01-04', '2023-01-05']),
    'Metric_A': [10, 12, np.nan, 15, 16],
    'Metric_B': [5, np.nan, 7, 8, np.nan]
}

df = pd.DataFrame(data)
print('Original:')
print(df)

df_ffill = df.fillna(method='ffill')
print('After ffill:')
print(df_ffill)


Original:
        Date  Metric_A  Metric_B
0 2023-01-01      10.0       5.0
1 2023-01-02      12.0       NaN
2 2023-01-03       NaN       7.0
3 2023-01-04      15.0       8.0
4 2023-01-05      16.0       NaN
After ffill:
        Date  Metric_A  Metric_B
0 2023-01-01      10.0       5.0
1 2023-01-02      12.0       5.0
2 2023-01-03      12.0       7.0
3 2023-01-04      15.0       8.0
4 2023-01-05      16.0       8.0


/tmp/ipython-input-2482268706.py:14: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df_ffill = df.fillna(method='ffill')


### 2.4 Backward fill (bfill) — fill from the next observed value

In [10]:
import pandas as pd
import numpy as np

data = {
    'Date': pd.to_datetime(['2023-01-01', '2023-01-02', '2023-01-03', '2023-01-04', '2023-01-05']),
    'Metric_A': [10, 12, np.nan, 15, 16],
    'Metric_B': [5, np.nan, 7, 8, np.nan]
}

df = pd.DataFrame(data)
print('Original:')
print(df)

df_bfill = df.fillna(method='bfill')
print('After bfill:')
print(df_bfill)


Original:
        Date  Metric_A  Metric_B
0 2023-01-01      10.0       5.0
1 2023-01-02      12.0       NaN
2 2023-01-03       NaN       7.0
3 2023-01-04      15.0       8.0
4 2023-01-05      16.0       NaN
After bfill:
        Date  Metric_A  Metric_B
0 2023-01-01      10.0       5.0
1 2023-01-02      12.0       7.0
2 2023-01-03      15.0       7.0
3 2023-01-04      15.0       8.0
4 2023-01-05      16.0       NaN


/tmp/ipython-input-1561499104.py:14: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df_bfill = df.fillna(method='bfill')


### 2.5 Handle duplicates (duplicated / drop_duplicates)

In [11]:
import pandas as pd

data = {
    'Employee ID': [1, 2, 3, 1, 4, 3],
    'Name': ['Khalid', 'Layla', 'Ahmed', 'Khalid', 'Fatimah', 'Ahmed'],
    'Department': ['IT', 'HR', 'Finance', 'IT', 'Marketing', 'Finance']
}

df = pd.DataFrame(data)
print('Original:')
print(df)

# Detect fully duplicated rows
duplicate_rows = df[df.duplicated()]
print('Detected duplicates (fully duplicated rows):')
print(duplicate_rows)

# Remove exact duplicates
df_no_duplicates = df.drop_duplicates()
print('After drop_duplicates():')
print(df_no_duplicates)


Original:
   Employee ID     Name Department
0            1   Khalid         IT
1            2    Layla         HR
2            3    Ahmed    Finance
3            1   Khalid         IT
4            4  Fatimah  Marketing
5            3    Ahmed    Finance
Detected duplicates (fully duplicated rows):
   Employee ID    Name Department
3            1  Khalid         IT
5            3   Ahmed    Finance
After drop_duplicates():
   Employee ID     Name Department
0            1   Khalid         IT
1            2    Layla         HR
2            3    Ahmed    Finance
4            4  Fatimah  Marketing


### 2.6 Drop duplicates based on a subset of columns

In [13]:
import pandas as pd

data = {
    'Employee ID': [1, 2, 3, 1, 4, 3, 5],
    'Name': ['Omar', 'Khalid', 'Layla', 'Ahmed', 'Khalid', 'Fatimah', 'Ahmed'],
    'Department': ['IT', 'HR', 'Finance', 'IT', 'Marketing', 'Finance', 'HR']
}

df = pd.DataFrame(data)
print('Original:')
print(df)

# Consider a row duplicated only if (Employee ID, Department) repeats
df_no_duplicates_subset = df.drop_duplicates(subset=['Employee ID', 'Department'])
print("After drop_duplicates(subset=['Employee ID', 'Department']):")
print(df_no_duplicates_subset)

Original:
   Employee ID     Name Department
0            1     Omar         IT
1            2   Khalid         HR
2            3    Layla    Finance
3            1    Ahmed         IT
4            4   Khalid  Marketing
5            3  Fatimah    Finance
6            5    Ahmed         HR
After drop_duplicates(subset=['Employee ID', 'Department']):
   Employee ID    Name Department
0            1    Omar         IT
1            2  Khalid         HR
2            3   Layla    Finance
4            4  Khalid  Marketing
6            5   Ahmed         HR


### 2.7 Keep the last duplicate occurrence (keep='last')

In [16]:
import pandas as pd

data = {
    'Employee ID': [1, 2, 3, 1, 4, 3],
    'Name': ['Khalid', 'Layla', 'Ahmed', 'Khalid', 'Fatimah', 'Ahmed'],
    'Department': ['IT', 'HR', 'Finance', 'IT', 'Marketing', 'Finance']
}

df = pd.DataFrame(data)
print('Original:')
print(df)

df_no_duplicates_last = df.drop_duplicates(keep='last')
print("After drop_duplicates(keep='last')")
print(df_no_duplicates_last)

Original:
   Employee ID     Name Department
0            1   Khalid         IT
1            2    Layla         HR
2            3    Ahmed    Finance
3            1   Khalid         IT
4            4  Fatimah  Marketing
5            3    Ahmed    Finance
After drop_duplicates(keep='last')
   Employee ID     Name Department
1            2    Layla         HR
3            1   Khalid         IT
4            4  Fatimah  Marketing
5            3    Ahmed    Finance


## 3) Data-type conversion and standardization

### 3.1 Convert numeric strings to numbers (pd.to_numeric, errors='coerce')

In [19]:
import pandas as pd
import numpy as np

data = {
    'Employee ID': [1, 2, 3, 4],
    'Salary_Str': ['50000', '60000', '75000', 'Not available'],
    'Bonus_Str': ['10000', '12000', '5000', 'Unknown']
}

df = pd.DataFrame(data)
print('dtypes (before):')
print(df.dtypes)
print('Original:')
print(df)

df['Salary'] = pd.to_numeric(df['Salary_Str'], errors='coerce')
df['Bonus'] = pd.to_numeric(df['Bonus_Str'], errors='coerce')

print('dtypes (after):')
print(df.dtypes)
print('After conversion:')
print(df)


dtypes (before):
Employee ID     int64
Salary_Str     object
Bonus_Str      object
dtype: object
Original:
   Employee ID     Salary_Str Bonus_Str
0            1          50000     10000
1            2          60000     12000
2            3          75000      5000
3            4  Not available   Unknown
dtypes (after):
Employee ID      int64
Salary_Str      object
Bonus_Str       object
Salary         float64
Bonus          float64
dtype: object
After conversion:
   Employee ID     Salary_Str Bonus_Str   Salary    Bonus
0            1          50000     10000  50000.0  10000.0
1            2          60000     12000  60000.0  12000.0
2            3          75000      5000  75000.0   5000.0
3            4  Not available   Unknown      NaN      NaN


### 3.2 Convert string dates to datetime (pd.to_datetime, format='mixed', dayfirst=True)

In [20]:
import pandas as pd

data = {
    'Employee ID': [1, 2, 3, 4],
    'Hire Date_Str': ['15-01-2020', '03-20-2019', '2021-10-05', '01-07-2018']
}

df = pd.DataFrame(data)
print('Original:')
print(df)

df['Hire Date'] = pd.to_datetime(df['Hire Date_Str'], format='mixed', dayfirst=True)
print('After pd.to_datetime:')
print(df)
print('dtypes:')
print(df.dtypes)


Original:
   Employee ID Hire Date_Str
0            1    15-01-2020
1            2    03-20-2019
2            3    2021-10-05
3            4    01-07-2018
After pd.to_datetime:
   Employee ID Hire Date_Str  Hire Date
0            1    15-01-2020 2020-01-15
1            2    03-20-2019 2019-03-20
2            3    2021-10-05 2021-10-05
3            4    01-07-2018 2018-07-01
dtypes:
Employee ID               int64
Hire Date_Str            object
Hire Date        datetime64[ns]
dtype: object


### 3.3 Rename columns (df.rename)

In [21]:
import pandas as pd

data = {
    'Emp ID': [1, 2, 3],
    'Emp Name': ['Ahmed', 'Khalid', 'Layla'],
    'Dept': ['IT', 'HR', 'Finance']
}

df = pd.DataFrame(data)
print('Original:')
print(df)

df_renamed = df.rename(columns={'Emp ID': 'Employee ID', 'Emp Name': 'Employee Name', 'Dept': 'Department'})
print('After rename:')
print(df_renamed)


Original:
   Emp ID Emp Name     Dept
0       1    Ahmed       IT
1       2   Khalid       HR
2       3    Layla  Finance
After rename:
   Employee ID Employee Name Department
0            1         Ahmed         IT
1            2        Khalid         HR
2            3         Layla    Finance


## 4) Feature engineering

### 4.1 Total compensation = salary + bonus

In [22]:
import pandas as pd

data = {
    'Employee ID': [1, 2, 3, 4],
    'Salary': [50000, 60000, 75000, 55000],
    'Bonus': [10000, 12000, 15000, 11000]
}

df = pd.DataFrame(data)
print('Original:')
print(df)

df['Total Compensation'] = df['Salary'] + df['Bonus']
print('After adding Total Compensation:')
print(df)


Original:
   Employee ID  Salary  Bonus
0            1   50000  10000
1            2   60000  12000
2            3   75000  15000
3            4   55000  11000
After adding Total Compensation:
   Employee ID  Salary  Bonus  Total Compensation
0            1   50000  10000               60000
1            2   60000  12000               72000
2            3   75000  15000               90000
3            4   55000  11000               66000


### 4.2 Categorize performance using np.where

In [23]:
import pandas as pd
import numpy as np

data = {
    'Employee ID': [1, 2, 3, 4, 5],
    'Performance Rating': [4, 2, 5, 3, 1]
}

df = pd.DataFrame(data)
print('Original:')
print(df)

df['Performance Category'] = np.where(
    df['Performance Rating'] >= 4,
    'High Performer',
    'Average/Low Performer'
)

print('After categorization:')
print(df)


Original:
   Employee ID  Performance Rating
0            1                   4
1            2                   2
2            3                   5
3            4                   3
4            5                   1
After categorization:
   Employee ID  Performance Rating   Performance Category
0            1                   4         High Performer
1            2                   2  Average/Low Performer
2            3                   5         High Performer
3            4                   3  Average/Low Performer
4            5                   1  Average/Low Performer


### 4.3 Extract year/month/day from a datetime column (.dt)

In [24]:
import pandas as pd

data = {
    'Employee ID': [1, 2, 3],
    'Hire Date': pd.to_datetime(['2020-01-15', '2019-03-20', '2021-10-05'])
}

df = pd.DataFrame(data)
print('Original:')
print(df)

df['Hire Year'] = df['Hire Date'].dt.year
df['Hire Month'] = df['Hire Date'].dt.month
df['Hire Day'] = df['Hire Date'].dt.day

print('After extraction:')
print(df)


Original:
   Employee ID  Hire Date
0            1 2020-01-15
1            2 2019-03-20
2            3 2021-10-05
After extraction:
   Employee ID  Hire Date  Hire Year  Hire Month  Hire Day
0            1 2020-01-15       2020           1        15
1            2 2019-03-20       2019           3        20
2            3 2021-10-05       2021          10         5


### 4.4 Years of service (tenure) from hire date

In [25]:
import pandas as pd
from datetime import datetime

data = {
    'Employee ID': [1, 2, 3],
    'Hire Date': pd.to_datetime(['2020-01-15', '2018-07-01', '2022-04-10'])
}

df = pd.DataFrame(data)
print('Original:')
print(df)

# Reference date (could be 'today' or any analysis date)
current_date = datetime(2025, 10, 15)

# Tenure in years
df['Years of Service'] = (current_date - df['Hire Date']).dt.days / 365.25

print('After Years of Service:')
print(df.round(1))


Original:
   Employee ID  Hire Date
0            1 2020-01-15
1            2 2018-07-01
2            3 2022-04-10
After Years of Service:
   Employee ID  Hire Date  Years of Service
0            1 2020-01-15               5.7
1            2 2018-07-01               7.3
2            3 2022-04-10               3.5


## 5) Outlier detection and treatment

### 5.1 Detect outliers using IQR (Q1/Q3)

In [26]:
import pandas as pd
import numpy as np

data = {
    'Employee ID': list(range(1, 11)),
    'Salary': [50000, 55000, 60000, 62000, 65000, 70000, 72000, 75000, 80000, 200000]
}

df = pd.DataFrame(data)
print('Original:')
print(df)

Q1 = df['Salary'].quantile(0.25)
Q3 = df['Salary'].quantile(0.75)
IQR = Q3 - Q1

lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR

outliers = df[(df['Salary'] < lower_bound) | (df['Salary'] > upper_bound)]
print('Outliers (IQR method):')
print(outliers)
print(f"Lower bound: {lower_bound:.2f}, Upper bound: {upper_bound:.2f}")


Original:
   Employee ID  Salary
0            1   50000
1            2   55000
2            3   60000
3            4   62000
4            5   65000
5            6   70000
6            7   72000
7            8   75000
8            9   80000
9           10  200000
Outliers (IQR method):
   Employee ID  Salary
9           10  200000
Lower bound: 39875.00, Upper bound: 94875.00


### 5.2 Cap outliers (capping) using the same bounds

In [27]:
import pandas as pd
import numpy as np

data = {
    'Employee ID': list(range(1, 11)),
    'Salary': [50000, 55000, 60000, 62000, 65000, 70000, 72000, 75000, 80000, 200000]
}

df = pd.DataFrame(data)

Q1 = df['Salary'].quantile(0.25)
Q3 = df['Salary'].quantile(0.75)
IQR = Q3 - Q1

lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR

# Capping: create a capped copy
df['Salary_Capped'] = df['Salary'].copy()
df.loc[df['Salary_Capped'] < lower_bound, 'Salary_Capped'] = lower_bound
df.loc[df['Salary_Capped'] > upper_bound, 'Salary_Capped'] = upper_bound

print(df)


   Employee ID  Salary  Salary_Capped
0            1   50000          50000
1            2   55000          55000
2            3   60000          60000
3            4   62000          62000
4            5   65000          65000
5            6   70000          70000
6            7   72000          72000
7            8   75000          75000
8            9   80000          80000
9           10  200000          94875


## 6) Aggregation & group analysis

### 6.1 Employee count by department (groupby + size)

In [28]:
import pandas as pd

data = {
    'Employee ID': [1, 2, 3, 4, 5, 6],
    'Department': ['IT', 'HR', 'IT', 'Finance', 'HR', 'Finance'],
    'Job Title': ['Analyst', 'Specialist', 'Developer', 'Accountant', 'Manager', 'Analyst'],
    'Salary': [70000, 60000, 72000, 65000, 80000, 68000]
}

df = pd.DataFrame(data)

employee_count_by_dept = df.groupby('Department').size().reset_index(name='Employee Count')
print(employee_count_by_dept)


  Department  Employee Count
0    Finance               2
1         HR               2
2         IT               2


### 6.2 Average salary by (Department, Job Title)

In [29]:
import pandas as pd

data = {
    'Employee ID': [1, 2, 3, 4, 5, 6],
    'Department': ['IT', 'HR', 'IT', 'Finance', 'HR', 'Finance'],
    'Job Title': ['Analyst', 'Specialist', 'Developer', 'Accountant', 'Manager', 'Analyst'],
    'Salary': [70000, 60000, 72000, 65000, 80000, 68000]
}

df = pd.DataFrame(data)

avg_salary_by_dept_job = df.groupby(['Department', 'Job Title'])['Salary'].mean().reset_index()
print(avg_salary_by_dept_job)


  Department   Job Title   Salary
0    Finance  Accountant  65000.0
1    Finance     Analyst  68000.0
2         HR     Manager  80000.0
3         HR  Specialist  60000.0
4         IT     Analyst  70000.0
5         IT   Developer  72000.0


### 6.3 Multiple aggregation metrics with agg

In [30]:
import pandas as pd

data = {
    'Employee ID': [1, 2, 3, 4, 5, 6],
    'Department': ['IT', 'HR', 'IT', 'Finance', 'HR', 'Finance'],
    'Salary': [70000, 60000, 72000, 65000, 80000, 68000]
}

df = pd.DataFrame(data)

aggregated_data = df.groupby('Department').agg(
    Average_Salary=('Salary', 'mean'),
    Max_Salary=('Salary', 'max'),
    Min_Salary=('Salary', 'min'),
    Employee_Count=('Employee ID', 'count')
).reset_index()

print(aggregated_data)


  Department  Average_Salary  Max_Salary  Min_Salary  Employee_Count
0    Finance         66500.0       68000       65000               2
1         HR         70000.0       80000       60000               2
2         IT         71000.0       72000       70000               2


### 6.4 Pivot table for cross-tab analysis (pd.pivot_table)

In [31]:
import pandas as pd

data = {
    'Employee ID': [1, 2, 3, 4, 5, 6],
    'Department': ['IT', 'HR', 'IT', 'Finance', 'HR', 'Finance'],
    'Job Title': ['Analyst', 'Specialist', 'Developer', 'Accountant', 'Manager', 'Analyst'],
    'Salary': [70000, 60000, 72000, 65000, 80000, 68000]
}

df = pd.DataFrame(data)

pivot_table_data = pd.pivot_table(
    df,
    values='Salary',
    index='Department',
    columns='Job Title',
    aggfunc='mean'
)

print(pivot_table_data)


Job Title   Accountant  Analyst  Developer  Manager  Specialist
Department                                                     
Finance        65000.0  68000.0        NaN      NaN         NaN
HR                 NaN      NaN        NaN  80000.0     60000.0
IT                 NaN  70000.0    72000.0      NaN         NaN
